<a href="https://colab.research.google.com/github/kznts9v-1lya/formal-lang-course/blob/task5-rpq-research/researches/rpq_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>